# Semantic Kernel with OpenBnB MCP Server Integration

এই নোটবুকটি দেখায় কীভাবে Semantic Kernel ব্যবহার করে প্রকৃত OpenBnB MCP সার্ভারের সাথে MCPStdioPlugin ব্যবহার করে আসল Airbnb থাকার জায়গা অনুসন্ধান করা যায়। LLM অ্যাক্সেসের জন্য এটি Microsoft Foundry ব্যবহার করে। আপনার পরিবেশ ভেরিয়েবলগুলি সেট আপ করতে, আপনি [Setup Lesson ](/00-course-setup/README.md) অনুসরণ করতে পারেন।


## প্রয়োজনীয় প্যাকেজগুলি আমদানি করুন


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP প্লাগইন সংযোগ তৈরি করা

আমরা MCPStdioPlugin ব্যবহার করে [OpenBnB MCP সার্ভার](https://github.com/openbnb-org/mcp-server-airbnb) এর সাথে সংযোগ করব। এই সার্ভারটি @openbnb/mcp-server-airbnb প্যাকেজের মাধ্যমে Airbnb অনুসন্ধানের কার্যকারিতা প্রদান করে।


## ক্লায়েন্ট তৈরি করা

এই নমুনায়, আমরা আমাদের LLM অ্যাক্সেসের জন্য Microsoft Foundry ব্যবহার করব। নিশ্চিত করুন যে আপনার পরিবেশ ভেরিয়েবলগুলি সঠিকভাবে সেট আপ করা হয়েছে।


## Environment Configuration

Azure OpenAI সেটিংস কনফিগার করুন। নিশ্চিত করুন যে আপনার কাছে নিম্নলিখিত পরিবেশ ভেরিয়েবলগুলি সেট করা আছে:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP ইন্টিগ্রেশন বোঝা

এই নোটবুকটি **বাস্তব OpenBnB MCP সার্ভার** এর সাথে সংযুক্ত যা প্রকৃত Airbnb সার্চ ফাংশনালিটি প্রদান করে।

### এটি কীভাবে কাজ করে:

1. **MCPStdioPlugin**: MCP সার্ভারের সাথে স্ট্যান্ডার্ড ইনপুট/আউটপুট যোগাযোগ ব্যবহার করে
2. **বাস্তব NPM প্যাকেজ**: npx এর মাধ্যমে `@openbnb/mcp-server-airbnb` ডাউনলোড এবং চালায়
3. **লাইভ ডেটা**: তাঁদের API থেকে প্রকৃত Airbnb সম্পত্তির তথ্য প্রদান করে
4. **ফাংশন আবিষ্কার**: এজেন্ট স্বয়ংক্রিয়ভাবে MCP সার্ভার থেকে উপলব্ধ ফাংশন গুলো আবিষ্কার করে

### উপলব্ধ ফাংশনসমূহ:

OpenBnB MCP সার্ভার সাধারণত প্রদান করে:
- **search_listings** - অবস্থান এবং মানদণ্ড অনুসারে Airbnb সম্পত্তি অনুসন্ধান
- **get_listing_details** - নির্দিষ্ট সম্পত্তির বিস্তারিত তথ্য পাবার জন্য
- **check_availability** - নির্দিষ্ট তারিখগুলিতে উপলব্ধতা পরীক্ষা করা
- **get_reviews** - সম্পত্তির জন্য রিভিউ সংগ্রহ করা
- **get_host_info** - সম্পত্তির মালিকদের সম্পর্কে তথ্য পাওয়া

### পূর্বশর্তসমূহ:

- আপনার সিস্টেমে **Node.js** ইনস্টল করা আছে
- MCP সার্ভার প্যাকেজ ডাউনলোডের জন্য **ইন্টারনেট সংযোগ**
- **NPX** উপলব্ধ (Node.js সহ আসে)

### সংযোগ পরীক্ষা:

আপনি ম্যানুয়ালি MCP সার্ভারটি চালিয়ে পরীক্ষা করতে পারেন:
```bash
npx -y @openbnb/mcp-server-airbnb
```

এটি OpenBnB MCP সার্ভার ডাউনলোড ও চালু করবে, যা Semantic Kernel প্রকৃত Airbnb ডেটার জন্য সংযুক্ত হয়।


## OpenBnB MCP সার্ভারের সাথে এজেন্ট চালানো

এখন আমরা AI এজেন্ট চালাব যা OpenBnB MCP সার্ভারের সাথে সংযুক্ত হয়ে স্টকহোমে ২ জন প্রাপ্তবয়স্ক এবং ১ জন শিশুর জন্য বাস্তব Airbnb থাকার জায়গার সন্ধান করবে। অনুসন্ধানের শর্তাবলী পরিবর্তন করতে `user_inputs` তালিকা পরিবর্তন করতে পারেন।


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# সারাংশ
অভিনন্দন! আপনি সফলভাবে একটি এআই এজেন্ট তৈরি করেছেন যা মডেল কনটেক্সট প্রোটোকল (MCP) ব্যবহার করে বাস্তব বিশ্বের আবাসনের অনুসন্ধানের সাথে সংযুক্ত:

## ব্যবহৃত প্রযুক্তিগুলো:
- সেমানটিক কার্নেল - Azure OpenAI সহ বুদ্ধিমান এজেন্ট তৈরির জন্য
- Microsoft Foundry - LLM সক্ষমতা এবং চ্যাট সম্পূর্ণকরণের জন্য
- MCP (Model Context Protocol) - স্ট্যান্ডার্ডাইজড টুল ইন্টিগ্রেশনের জন্য
- OpenBnB MCP সার্ভার - আসল Airbnb অনুসন্ধানের কার্যকারিতা জন্য
- Node.js/NPX - বাহ্যিক MCP সার্ভার চালানোর জন্য

## আপনি যা শিখেছেন:
- MCP ইন্টিগ্রেশন: সেমানটিক কার্নেল এজেন্টদের বহিরাগত MCP সার্ভারের সাথে সংযোগ করা
- রিয়েল-টাইম ডেটা অ্যাকসেস: লাইভ API গুলোর মাধ্যমে আসল Airbnb সম্পত্তি অনুসন্ধান
- প্রোটোকল কমিউনিকেশন: এজেন্ট এবং MCP সার্ভারের মধ্যে stdio যোগাযোগ ব্যবহার
- ফাংশন আবিষ্কার: MCP সার্ভার থেকে উপলব্ধ ফাংশন স্বয়ংক্রিয়ভাবে আবিষ্কার করা
- স্ট্রিমিং উত্তর: বাস্তব সময়ে ফাংশন কলগুলি ক্যাপচার এবং লগিং করা
- HTML রেন্ডারিং: স্টাইল করা টেবিল এবং ইন্টারেক্টিভ প্রদর্শনের মাধ্যমে এজেন্টের প্রতিক্রিয়া ফরম্যাট করা

## পরবর্তী ধাপসমূহ:
- অতিরিক্ত MCP সার্ভার একত্রীকরণ (আবহাওয়া, ফ্লাইট, রেস্টুরেন্ট)
- MCP এবং A2A প্রোটোকল যুক্ত একটি মাল্টি-এজেন্ট সিস্টেম তৈরি
- আপনার নিজস্ব ডেটা উৎসের জন্য কাস্টম MCP সার্ভার তৈরি করা
- সেশন জুড়ে স্থায়ী কথোপকথন স্মৃতি বাস্তবায়ন
- MCP সার্ভার অর্কেস্ট্রেশনের সাথে Azure Functions এ এজেন্ট মোতায়েন করা
- ব্যবহারকারী প্রমাণীকরণ এবং বুকিং সক্ষমতা যোগ করা


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**বিজ্ঞপ্তি**:  
এই ডকুমেন্টটি AI অনুবাদ সেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনূদিত হয়েছে। আমরা যথাসাধ্য সঠিকতার চেষ্টা করি, তবে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। প্রামাণিক উৎস হিসেবে মূল ভাষায় থাকা ডকুমেন্টটিকেই বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানুষের অনুবাদ গ্রহণ করার পরামর্শ দেওয়া হয়। এই অনুবাদের ব্যবহার থেকে সৃষ্ট কোনো ভুলবোঝাবুঝি বা ব্যাখ্যাগত ত্রুটির জন্য আমরা দায়বদ্ধ নই।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
